**As a solution for MRNET problem, we have made up 2 approaches but both are common in the fact that they consists of 9 different models. So we need an Ensemble method to combine those 9 Models and get an actual answer for the problem !**

In [0]:
#imports cell 
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import load_model
from tensorflow import keras
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense,Input,Flatten,GlobalAveragePooling2D,Dropout,InputLayer,GlobalAveragePooling3D,Reshape,BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import regularizers
from tensorflow.keras import optimizers
K.set_image_data_format("channels_first")

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
def load_validation_interrpolation_data(plane,label):
  #feature extractor
  base_model = VGG16(weights='imagenet',input_shape=(3,256,256),include_top=False,input_tensor=Input(shape=(3,256,256)))
  #Global Average Pooling
  inputs=Input(shape=(512,8,8))
  gap=GlobalAveragePooling2D()(inputs)
  model_gap=Model(inputs=inputs,outputs=gap)
  valid_X = np.load('/content/gdrive/My Drive/Dataset/Transformations/interpolated24_valid_' + plane + '.npy')
  val_X_2 = np.load('/content/gdrive/My Drive/Dataset/Transformations/mid3_valid_'+plane+'.npy') 
  valid_X=np.stack([valid_X]*3,axis=2)
  valid_Y = np.genfromtxt('/content/gdrive/My Drive/Dataset/MRNet-v1.0/valid-' + label + '.csv',delimiter=',')[:,1]
  new_valid_X = np.zeros(shape=(120,24,512))
  for i in range(120):
    first_output=base_model.predict(valid_X[i])
    new_valid_X[i] = model_gap.predict(first_output)
  valid_X = None
  new_valid_X=new_valid_X.reshape(120,512,24,1)
  return new_valid_X,val_X_2,valid_Y

  
def eval_model(label):
  #define your models.
  axial_model = load_model("/content/gdrive/My Drive/Models/PaperImplementation/axial_"+label+".h5")
  coronal_model = load_model("/content/gdrive/My Drive/Models/PaperImplementation/coronal_"+label+".h5")
  sagittal_model = load_model("/content/gdrive/My Drive/Models/PaperImplementation/sagittal_"+label+".h5")
  axial_model_TL = load_model("/content/gdrive/My Drive/Models/TransferLearning/axial_"+label+".h5")
  coronal_model_TL = load_model("/content/gdrive/My Drive/Models/TransferLearning/coronal_"+label+".h5")
  sagittal_model_TL = load_model("/content/gdrive/My Drive/Models/TransferLearning/sagittal_"+label+".h5")
  
  #load and predict on data.
  val_X,val_X_TL,val_y = load_validation_interrpolation_data('axial',label)
  w1 = axial_model.evaluate(val_X,val_y,verbose =0)[1]
  axial_preds = (axial_model.predict(val_X) > 0.5).astype(np.int_).ravel()
  w2 = axial_model_TL.evaluate(val_X_TL,val_y,verbose =0)[1]
  axial_preds_TL = (axial_model_TL.predict(val_X_TL) > 0.5).astype(np.int_).ravel()
  
  
  val_X,val_X_TL,val_y = load_validation_interrpolation_data('coronal',label)
  w3 = coronal_model.evaluate(val_X,val_y,verbose =0)[1]
  coronal_preds = (coronal_model.predict(val_X) > 0.5).astype(np.int_).ravel()
  w4 = coronal_model_TL.evaluate(val_X_TL,val_y,verbose =0)[1]
  coronal_preds_TL = (coronal_model_TL.predict(val_X_TL) > 0.5).astype(np.int_).ravel()
  
  
  val_X,val_X_TL,val_y = load_validation_interrpolation_data('sagittal',label)
  w5 = sagittal_model.evaluate(val_X,val_y,verbose =0)[1]
  sagittal_preds = (sagittal_model.predict(val_X) > 0.5).astype(np.int_).ravel()
  w6 = sagittal_model_TL.evaluate(val_X_TL,val_y,verbose =0)[1]
  sagittal_preds_TL = (sagittal_model_TL.predict(val_X_TL) > 0.5).astype(np.int_).ravel()
  
  #intialize main vector to store predicitions.
  preds = np.arange(len(axial_preds))
  weights = np.array([w1,w2,w3,w4,w5,w6])
  for i in range(len(axial_preds)):
    preds[i] = np.argmax(np.bincount(np.array([axial_preds[i],coronal_preds[i],
                                               sagittal_preds[i],axial_preds_TL[i],
                                               coronal_preds_TL[i],sagittal_preds_TL[i]]),weights))
    
  return accuracy_score(val_y,preds)
   

# Ensemble Weighted Majority Voting

In [19]:
eval_model('abnormal')

0.8916666666666667

In [20]:
eval_model('acl')

0.8

In [21]:
eval_model('meniscus')

0.7166666666666667